In [28]:
import tweepy
import os
import config

consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret
bearer_token = config.bearer_token

client = tweepy.Client(bearer_token=bearer_token)

query = "covid lang:en"

response = client.search_recent_tweets(query=query,tweet_fields=['attachments','author_id','context_annotations','conversation_id','created_at','entities','geo,id','in_reply_to_user_id','lang','possibly_sensitive','public_metrics','referenced_tweets','reply_settings','source','text','withheld'],user_fields=['created_at','description','entities,id','location','name','pinned_tweet_id','profile_image_url','protected,public_metrics','url','username','verified','withheld'],expansions=['attachments.poll_ids','attachments.media_keys','author_id','geo.place_id','in_reply_to_user_id','referenced_tweets.id','entities.mentions.username','referenced_tweets.id.author_id'],max_results=100)

# users = {u['id']: u for u in response.includes['users']}         
# for tweet in response.data:  
#     if users[tweet.author_id]:
#         user = users[tweet.author_id]
#     print(user.name)
#     print(user.username)
#     ref_tweets = tweet.referenced_tweets
#     if tweet.referenced_tweets:
#         print(tweet.referenced_tweets)
#         print(type(tweet.referenced_tweets))
#         print(len(tweet.referenced_tweets))

reftweet = client.get_tweet('1484932208427163650')
print(reftweet.data)



This child from Missouri is severely ill from COVID. The red ECMO tubes from his neck allow blood to bypass his heart &amp; damaged lungs. His windpipe was cut to place him on breathing machine longterm. He exercises to gain strength #TheySurvivedCovid 
https://t.co/ZCRJkggENL
